# Simulating Circuits for DM4 and DL5

This notebook provides cells that run different circuits; the results of the simulations can be explored with the CircuitVisualization notebook.
## Setup

In [ ]:
import networkx as nx
from neurokernel.LPU.InputProcessors.StepInputProcessor import StepInputProcessor
import numpy as np
name = 'DM4_step1_'
main_path = '/mnt/server-home/mehmet/alclustering/'
G_path = main_path + '{}.gexf'.format(name)
visual_components = np.load(main_path + 'visual_components_{}.npy'.format(name), allow_pickle=True)
visual_neurons = np.load(main_path + 'visual_neurons_{}.npy'.format(name), allow_pickle=True).item()
def large_scale_sim(name, G_path, conc = 1., DM4_b = 2.17 * 1e-2, DM4_d = 2.94, DL5_b = 0.1 * 2.94, exp_name = '', LN_3=False, LN_3_I = 10.):
    # Simulation function specifically to run DM4 and DL5 graphs. 
    G = nx.read_gexf(G_path)
    G = nx.MultiDiGraph(G)
    sim_name = name + '_conc_' + str(conc) + exp_name
    def set_br(G):
        for node in G.nodes():
            if '_OTP' in node:
                if 'DC1' in node:
                    G.nodes(data=True)[node]['br'] = 0.
                elif 'DM4' in node:
                    G.nodes(data=True)[node]['br'] = DM4_b
                    G.nodes(data=True)[node]['dr'] = DM4_d
                    # print('found DM4')
                else:
                    # G.nodes(data=True)[node]['br'] = 0.01
                    G.nodes(data=True)[node]['br'] = DL5_b
                    G.nodes(data=True)[node]['dr'] = DM4_d
        return G

    G = set_br(G)
    inputs = [i for i in G.nodes() if '_OTP' in i]
    fi = StepInputProcessor('conc', inputs, conc, start=1., stop=3.)
    print('Concentration Level:', conc)
    print('Inputs:', inputs)
    def simulate(G, t, inputs,
        record_var_list = None,
        sample_interval: int = 100,
    ):
        from neurokernel.LPU.LPU import LPU
        from neurokernel.LPU.InputProcessors.BaseInputProcessor import (
            BaseInputProcessor,
        )
        from neurokernel.LPU.InputProcessors.ArrayInputProcessor import (
            ArrayInputProcessor,
        )
        from neurokernel.LPU.OutputProcessors.OutputRecorder import OutputRecorder
        from neurokernel.LPU.OutputProcessors.FileOutputProcessor import FileOutputProcessor

        dt = t[1] - t[0]
        if isinstance(inputs, BaseInputProcessor):
            fi = [inputs]
        elif isinstance(inputs, (list, tuple, np.ndarray)) and isinstance(
            inputs[0], BaseInputProcessor
        ):
            fi = inputs
        elif isinstance(inputs, dict):
            for data in inputs.values():
                assert "uids" in data
                assert "data" in data
                assert isinstance(data["data"], np.ndarray)
            fi = [ArrayInputProcessor(inputs)]
        else:
            raise ValueError("Input not understood")
            
        if LN_3:
            fi += [StepInputProcessor('I', ['LN_3'], LN_3_I, start=1., stop=3.)]
            print('Adding current to LN3.')

        fo = OutputRecorder([('I', None), ('g', None)], sample_interval=sample_interval)
        fo2 = OutputRecorder([('spike_state', None)], sample_interval=1)
        fo3 = OutputRecorder([('V', None)], sample_interval=1)
        lpu = LPU(
            dt,
            "obj",
            G,
            device=0,
            id="EOS",
            input_processors=fi,
            output_processors=[fo, fo2, fo3],
            debug=False,
            manager=False
        )
        lpu.run(steps=len(t))

        fo.to_file('sim_results2/fo_{}'.format(sim_name))
        fo2.to_file('sim_results2/fo2_{}'.format(sim_name))
        fo3.to_file('sim_results2/fo3_{}'.format(sim_name))
        return fi, fo, fo2, fo3, lpu
    return simulate, G, fi



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm

class PSTH(object):
    """Peri-Stimulus Time Histogram
    This class facilitates computing PSTH of input spikes in an OOP fashion.
    For 2D spiking data, the PSTH is computed as an average across spike times
    for all neurons.
    .. seealso:: :py:func:`compute_psth`
    Parameters:
        spikes: 1D or 2D binary array of spike trains to compute PSTH from. For
            2D array, the temporal axis is infered as the longer dimension. The
            other dimension is treated as neuron indices.
        dt: time-step for spikes along the temporal dimension of :code:`spikes`
    """

    def __init__(
        self, spikes: np.ndarray, dt: float, window: float = 20e-3, shift: float = 10e-3
    ):
        self.window = window
        self.shift = shift
        self.dt = dt
        self.spikes = spikes
        self.psth, self.t = self.compute()

    def compute(self):
        spikes = self.spikes
        if len(spikes.shape) > 1:
            axis = int(spikes.shape[0] > spikes.shape[1])
            spikes = np.mean(spikes, axis=axis)

        cum_spikes = np.cumsum(spikes)

        duration = self.dt * len(cum_spikes)
        start = np.arange(0.0, duration - self.window, self.shift) // self.dt
        stop = np.arange(self.window, duration - self.dt, self.shift) // self.dt
        start = start.astype(int, copy=False)
        stop = stop.astype(int, copy=False)

        start = start[: len(stop)]

        rates = (cum_spikes[stop] - cum_spikes[start]) / self.window
        stamps = np.arange(0, len(rates) * self.shift - self.dt, self.shift)

        return rates, stamps

    def merge(self, others):
        if not hasattr(others, "__len__"):
            others = [others]
        for other in others:
            assert np.all(self.t == other.t)

        stack = [self.psth]
        for other in others:
            stack.append(other.psth)

        self.psth = np.vstack(stack)

def gen_step2_plots(fo, fo2, fo3, sim_name, filter_keyword = 'DM4', PN_filter_keyword = 'DM4'):
    tk = 1650
    plot_data = []
    back_plot_data = []
    interesting_components = []
    interesting_data = []
    sim_traces = {}
    

    fig = plt.figure(figsize=(15,15))
    fig.patch.set_facecolor('xkcd:black')

    f = plt.subplot(7,1,1)
    f.patch.set_facecolor('xkcd:black')
    data = fo3.get_output(var='V')
    PNs = [i for i in data.keys() if 'ORN' in i and filter_keyword in i]
    sim_traces['V_ORN'] = {}
    for PN in PNs[:]:
        plt.plot(data[PN]['time'], data[PN]['data'])
        sim_traces['V_ORN'][PN] = {'time': data[PN]['time'], 'data': data[PN]['data']}
    try:
        time_axis = data[PN]['time'].copy()
    except:
        return sim_traces
    plt.xlabel('Time [s]')
    plt.ylabel('Membrane Potential')
    ax = plt.gca()
    ax.spines['bottom'].set_color('white')
    ax.spines['top'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.spines['right'].set_color('white')
    ax.xaxis.label.set_color('white')
    ax.yaxis.label.set_color('white')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    ax.set_title('ORN Activity For {} ppm Acetone Delivery'.format(int(conc)), color='white')
    #ax.axvline(x=[time], color='white')


    f = plt.subplot(7,1,2)
    f.patch.set_facecolor('xkcd:black')
    data = fo3.get_output(var='V')
    PNs = [i for i in data.keys() if 'PN' in i and PN_filter_keyword in i]
    sim_traces['V_PN'] = {}
    for PN in PNs[:]:
        plt.plot(data[PN]['time'], data[PN]['data'])
        sim_traces['V_PN'][PN] = {'time': data[PN]['time'], 'data': data[PN]['data']}
    plt.xlabel('Time [s]')
    plt.ylabel('Membrane Potential')
    ax = plt.gca()
    ax.spines['bottom'].set_color('white')
    ax.spines['top'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.spines['right'].set_color('white')
    ax.xaxis.label.set_color('white')
    ax.yaxis.label.set_color('white')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    ax.set_title('PN Activity', color='white')
    #ax.axvline(x=[time], color='white')

    f = plt.subplot(7,1,3)
    f.patch.set_facecolor('xkcd:black')
    data = fo3.get_output(var='V')
    PNs = [i for i in data.keys() if 'LN' in i]
    sim_traces['V'] = {}
    if len(PNs)>0.:
        for PN in PNs[:]:
            plt.plot(data[PN]['time'], data[PN]['data'])
            sim_traces['V'][PN] = {'time': data[PN]['time'], 'data': data[PN]['data']}
        plt.xlabel('Time [s]')
        plt.ylabel('Membrane Potential')
        ax = plt.gca()
        ax.spines['bottom'].set_color('white')
        ax.spines['top'].set_color('white')
        ax.spines['left'].set_color('white')
        ax.spines['right'].set_color('white')
        ax.xaxis.label.set_color('white')
        ax.yaxis.label.set_color('white')
        ax.tick_params(axis='x', colors='white')
        ax.tick_params(axis='y', colors='white')
        ax.set_title('LN Activity', color='white')
        #ax.axvline(x=[time], color='white')

    f = plt.subplot(7,1,4)
    f.patch.set_facecolor('xkcd:black')
    data = fo.get_output(var='I')
    PNs = [i for i in data.keys() if '_AT' in i and 'PN' in i and filter_keyword in i]
    sim_traces['I'] = {}
    for i in range(len(PNs)):
        plt.plot(data[PNs[i]]['time'], data[PNs[i]]['data'])
        sim_traces['I'][PNs[i]] = {'time': data[PNs[i]]['time'], 'data': data[PNs[i]]['data']}
    plt.xlabel('Time [s]')
    plt.ylabel('Current')
    ax = plt.gca()
    ax.spines['bottom'].set_color('white')
    ax.spines['top'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.spines['right'].set_color('white')
    ax.xaxis.label.set_color('white')
    ax.yaxis.label.set_color('white')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    ax.set_title('Individual ORN>PN Synapse Current', color='white')
    #ax.axvline(x=[time], color='white')

    f = plt.subplot(7,1,5)
    f.patch.set_facecolor('xkcd:black')
    data = fo.get_output(var='I')
    PNs = [i for i in data.keys() if '_AT' in i and 'LN' in i and filter_keyword in i]
    if len(PNs)>0.:
        for i in range(len(PNs)):
            plt.plot(data[PNs[i]]['time'], data[PNs[i]]['data'])
        plt.xlabel('Time [s]')
        plt.ylabel('Current')
        ax = plt.gca()
        ax.spines['bottom'].set_color('white')
        ax.spines['top'].set_color('white')
        ax.spines['left'].set_color('white')
        ax.spines['right'].set_color('white')
        ax.xaxis.label.set_color('white')
        ax.yaxis.label.set_color('white')
        ax.tick_params(axis='x', colors='white')
        ax.tick_params(axis='y', colors='white')
        ax.set_title('Individual ORN>LN Synapse Current', color='white')
        #ax.axvline(x=[time], color='white')

    f = plt.subplot(7,1,6)
    f.patch.set_facecolor('xkcd:black')
    data = fo.get_output(var='I')
    PNs = [i for i in data.keys() if '_Converter' in i and 'LN' in i and PN_filter_keyword in i and 'PN' in i and not i.startswith(PN_filter_keyword)]
    if len(PNs)>0.:
        for i in range(len(PNs)):
            plt.plot(data[PNs[i]]['time'], data[PNs[i]]['data'])
        plt.xlabel('Time [s]')
        plt.ylabel('Current')
        ax = plt.gca()
        ax.spines['bottom'].set_color('white')
        ax.spines['top'].set_color('white')
        ax.spines['left'].set_color('white')
        ax.spines['right'].set_color('white')
        ax.xaxis.label.set_color('white')
        ax.yaxis.label.set_color('white')
        ax.tick_params(axis='x', colors='white')
        ax.tick_params(axis='y', colors='white')
        ax.set_title('Individual LN>PN Synapse Current', color='white')
        #ax.axvline(x=[time], color='white')

    f = plt.subplot(7,1,7)
    f.patch.set_facecolor('xkcd:black')
    data = fo.get_output(var='I')
    PNs = [i for i in data.keys() if '_Converter' in i and 'LN' in i and PN_filter_keyword in i and 'PN' in i and i.startswith(PN_filter_keyword)]
    if len(PNs)>0.:
        for i in range(len(PNs)):
            plt.plot(data[PNs[i]]['time'], data[PNs[i]]['data'])
        plt.xlabel('Time [s]')
        plt.ylabel('Current')
        ax = plt.gca()
        ax.spines['bottom'].set_color('white')
        ax.spines['top'].set_color('white')
        ax.spines['left'].set_color('white')
        ax.spines['right'].set_color('white')
        ax.xaxis.label.set_color('white')
        ax.yaxis.label.set_color('white')
        ax.tick_params(axis='x', colors='white')
        ax.tick_params(axis='y', colors='white')
        ax.set_title('Individual PN>LN Synapse Current', color='white')
        #ax.axvline(x=[time], color='white')


    plt.tight_layout()
    plt.savefig('{}_conc.png'.format(sim_name))
    plt.close()
    
    data = fo2.get_output(var='spike_state')
    PNs = [i for i in data.keys() if 'PN' in i and PN_filter_keyword in i]
    sim_traces['psth'] = {}
    sim_traces['spike_state'] = {}
    for PN in PNs[:]:
        
        spike_times = time_axis
        spike_data = spike_times * 0.
        real_spike_data = data[PN]['data']
        if data[PN]['data'].shape[0]>0:
            for i in range(time_axis.shape[0]):
                if np.min(np.abs(time_axis[i] - data[PN]['data']))<dt/2:
                    spike_data[i] = 1.
        psth_data = PSTH( spike_data, 1e-5, window=3e-1, shift=50e-3)
        psth_step = psth_data.psth
        psth_t = psth_data.t
        sim_traces['psth'][PN] = {'time': psth_t, 'data': psth_step}
        # plt.plot(psth_t, psth_step)
        sim_traces['spike_state'][PN] = {'time': real_spike_data}
    return sim_traces

# Single Glomerulus Simulations:

In [ ]:
import time
import gc
names = ['DL5_3_all']
simulation_traces = {}
for name in names:
    for DM4_b_coeff in [0.0]:
        for DL5_b_coeff in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
            time.sleep(1)
            conc = 10.
            G_path = main_path + '{}.gexf'.format(name)
            visual_components = np.load(main_path + 'visual_components_{}.npy'.format(name), allow_pickle=True)
            visual_neurons = np.load(main_path + 'visual_neurons_{}.npy'.format(name), allow_pickle=True).item()
            simulate, G, fi = large_scale_sim(name, G_path, conc=conc, DM4_b = DM4_b_coeff * 0.3, DM4_d = 2.94, DL5_b = DL5_b_coeff * 0.3, 
                                              exp_name = '_my_DL5_experiment_' + str(DM4_b_coeff) + '_' + str(DL5_b_coeff), LN_3=True, LN_3_I = 20.)
            dt = 1e-5
            dur = 4.
            steps = int((dur+dt/2)//dt)
            t = np.arange(steps)*dt
            fi, fo, fo2, fo3, lpu = simulate(G, t, fi)
            
names = ['DL5_3_ln1', 'DL5_3_ln12', 'DL5_3_ln2', 'DL5_3_lnless']
for name in names:
    for DM4_b_coeff in [0.0]:
        for DL5_b_coeff in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
            time.sleep(1)
            conc = 10.
            G_path = main_path + '{}.gexf'.format(name)
            visual_components = np.load(main_path + 'visual_components_{}.npy'.format(name), allow_pickle=True)
            visual_neurons = np.load(main_path + 'visual_neurons_{}.npy'.format(name), allow_pickle=True).item()
            simulate, G, fi = large_scale_sim(name, G_path, conc=conc, DM4_b = DM4_b_coeff * 0.3, DM4_d = 2.94, DL5_b = DL5_b_coeff * 0.3, 
                                              exp_name = '_my_DL5_experiment_' + str(DM4_b_coeff) + '_' + str(DL5_b_coeff), LN_3=False, LN_3_I = 20.)
            dt = 1e-5
            dur = 4.
            steps = int((dur+dt/2)//dt)
            t = np.arange(steps)*dt
            fi, fo, fo2, fo3, lpu = simulate(G, t, fi)

In [ ]:
import time
import gc
names = ['DL5_3i_all']
simulation_traces = {}
for name in names:
    for DM4_b_coeff in [0.0]:
        for DL5_b_coeff in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
            time.sleep(1)
            conc = 10.
            G_path = main_path + '{}.gexf'.format(name)
            visual_components = np.load(main_path + 'visual_components_{}.npy'.format(name), allow_pickle=True)
            visual_neurons = np.load(main_path + 'visual_neurons_{}.npy'.format(name), allow_pickle=True).item()
            simulate, G, fi = large_scale_sim(name, G_path, conc=conc, DM4_b = DM4_b_coeff * 0.3, DM4_d = 2.94, DL5_b = DL5_b_coeff * 0.3, 
                                              exp_name = '_my_DL5_experiment_' + str(DM4_b_coeff) + '_' + str(DL5_b_coeff), LN_3=True, LN_3_I = 20.)
            dt = 1e-5
            dur = 4.
            steps = int((dur+dt/2)//dt)
            t = np.arange(steps)*dt
            fi, fo, fo2, fo3, lpu = simulate(G, t, fi)
            
names = ['DL5_3i_ln1', 'DL5_3i_ln12', 'DL5_3i_ln2', 'DL5_3i_lnless']
for name in names:
    for DM4_b_coeff in [0.0]:
        for DL5_b_coeff in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
            time.sleep(1)
            conc = 10.
            G_path = main_path + '{}.gexf'.format(name)
            visual_components = np.load(main_path + 'visual_components_{}.npy'.format(name), allow_pickle=True)
            visual_neurons = np.load(main_path + 'visual_neurons_{}.npy'.format(name), allow_pickle=True).item()
            simulate, G, fi = large_scale_sim(name, G_path, conc=conc, DM4_b = DM4_b_coeff * 0.3, DM4_d = 2.94, DL5_b = DL5_b_coeff * 0.3, 
                                              exp_name = '_my_DL5_experiment_' + str(DM4_b_coeff) + '_' + str(DL5_b_coeff), LN_3=False, LN_3_I = 20.)
            dt = 1e-5
            dur = 4.
            steps = int((dur+dt/2)//dt)
            t = np.arange(steps)*dt
            fi, fo, fo2, fo3, lpu = simulate(G, t, fi)

In [ ]:
import time
import gc
names = ['DM4_3_all']
simulation_traces = {}
for name in names:
    for DM4_b_coeff in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        for DL5_b_coeff in [0.0]:
            time.sleep(1)
            conc = 10.
            G_path = main_path + '{}.gexf'.format(name)
            visual_components = np.load(main_path + 'visual_components_{}.npy'.format(name), allow_pickle=True)
            visual_neurons = np.load(main_path + 'visual_neurons_{}.npy'.format(name), allow_pickle=True).item()
            simulate, G, fi = large_scale_sim(name, G_path, conc=conc, DM4_b = DM4_b_coeff * 0.3, DM4_d = 2.94, DL5_b = DL5_b_coeff * 0.3, 
                                              exp_name = '_my_DM4_experiment_' + str(DM4_b_coeff) + '_' + str(DL5_b_coeff), LN_3=True, LN_3_I = 20.)
            dt = 1e-5
            dur = 4.
            steps = int((dur+dt/2)//dt)
            t = np.arange(steps)*dt
            fi, fo, fo2, fo3, lpu = simulate(G, t, fi)
            


In [ ]:
import time
import gc
names = ['DM4_3i_all']
simulation_traces = {}
for name in names:
    for DM4_b_coeff in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        for DL5_b_coeff in [0.0]:
            time.sleep(1)
            conc = 10.
            G_path = main_path + '{}.gexf'.format(name)
            visual_components = np.load(main_path + 'visual_components_{}.npy'.format(name), allow_pickle=True)
            visual_neurons = np.load(main_path + 'visual_neurons_{}.npy'.format(name), allow_pickle=True).item()
            simulate, G, fi = large_scale_sim(name, G_path, conc=conc, DM4_b = DM4_b_coeff * 0.3, DM4_d = 2.94, DL5_b = DL5_b_coeff * 0.3, 
                                              exp_name = '_my_DM4_experiment_' + str(DM4_b_coeff) + '_' + str(DL5_b_coeff), LN_3=True, LN_3_I = 20.)
            dt = 1e-5
            dur = 4.
            steps = int((dur+dt/2)//dt)
            t = np.arange(steps)*dt
            fi, fo, fo2, fo3, lpu = simulate(G, t, fi)
            
names = ['DM4_3_ln1', 'DM4_3_ln12', 'DM4_3_ln2', 'DM4_3_lnless']
for name in names:
    for DM4_b_coeff in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        for DL5_b_coeff in [0.0]:
            time.sleep(1)
            conc = 10.
            G_path = main_path + '{}.gexf'.format(name)
            visual_components = np.load(main_path + 'visual_components_{}.npy'.format(name), allow_pickle=True)
            visual_neurons = np.load(main_path + 'visual_neurons_{}.npy'.format(name), allow_pickle=True).item()
            simulate, G, fi = large_scale_sim(name, G_path, conc=conc, DM4_b = DM4_b_coeff * 0.3, DM4_d = 2.94, DL5_b = DL5_b_coeff * 0.3, 
                                              exp_name = '_my_DM4_experiment_' + str(DM4_b_coeff) + '_' + str(DL5_b_coeff), LN_3=False, LN_3_I = 20.)
            dt = 1e-5
            dur = 4.
            steps = int((dur+dt/2)//dt)
            t = np.arange(steps)*dt
            fi, fo, fo2, fo3, lpu = simulate(G, t, fi)

In [ ]:
import time
import gc
names = ['DM4_3i_all']
simulation_traces = {}
for name in names:
    for DM4_b_coeff in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        for DL5_b_coeff in [0.0]:
            time.sleep(1)
            conc = 10.
            G_path = main_path + '{}.gexf'.format(name)
            visual_components = np.load(main_path + 'visual_components_{}.npy'.format(name), allow_pickle=True)
            visual_neurons = np.load(main_path + 'visual_neurons_{}.npy'.format(name), allow_pickle=True).item()
            simulate, G, fi = large_scale_sim(name, G_path, conc=conc, DM4_b = DM4_b_coeff * 0.3, DM4_d = 2.94, DL5_b = DL5_b_coeff * 0.3, 
                                              exp_name = '_my_DM4_experiment_' + str(DM4_b_coeff) + '_' + str(DL5_b_coeff), LN_3=True, LN_3_I = 20.)
            dt = 1e-5
            dur = 4.
            steps = int((dur+dt/2)//dt)
            t = np.arange(steps)*dt
            fi, fo, fo2, fo3, lpu = simulate(G, t, fi)
            
names = ['DM4_3i_ln1', 'DM4_3i_ln12', 'DM4_3i_ln2', 'DM4_3i_lnless']
for name in names:
    for DM4_b_coeff in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        for DL5_b_coeff in [0.0]:
            time.sleep(1)
            conc = 10.
            G_path = main_path + '{}.gexf'.format(name)
            visual_components = np.load(main_path + 'visual_components_{}.npy'.format(name), allow_pickle=True)
            visual_neurons = np.load(main_path + 'visual_neurons_{}.npy'.format(name), allow_pickle=True).item()
            simulate, G, fi = large_scale_sim(name, G_path, conc=conc, DM4_b = DM4_b_coeff * 0.3, DM4_d = 2.94, DL5_b = DL5_b_coeff * 0.3, 
                                              exp_name = '_my_DM4_experiment_' + str(DM4_b_coeff) + '_' + str(DL5_b_coeff), LN_3=False, LN_3_I = 20.)
            dt = 1e-5
            dur = 4.
            steps = int((dur+dt/2)//dt)
            t = np.arange(steps)*dt
            fi, fo, fo2, fo3, lpu = simulate(G, t, fi)

In [ ]:
import time
import gc
names = ['DM4DL5_3_full']
simulation_traces = {}
for name in names:
    for DM4_b_coeff in [0.0,0.2,0.4,0.6,0.8,1.0]:
        for DL5_b_coeff in [0.0,0.2,0.4,0.6,0.8,1.0]:
            time.sleep(1)
            conc = 10.
            G_path = '/mnt/server-home/mehmet/alclustering/{}.gexf'.format(name)
            visual_components = np.load('/mnt/server-home/mehmet/alclustering/visual_components_{}.npy'.format(name), allow_pickle=True)
            visual_neurons = np.load('/mnt/server-home/mehmet/alclustering/visual_neurons_{}.npy'.format(name), allow_pickle=True).item()
            simulate, G, fi = large_scale_sim(name, G_path, conc=conc, DM4_b = DM4_b_coeff * 0.3, DM4_d = 2.94, DL5_b = DL5_b_coeff * 0.3, exp_name = '_my_experiment_' + str(DM4_b_coeff) + '_' + str(DL5_b_coeff))
            dt = 1e-5
            dur = 4.
            steps = int((dur+dt/2)//dt)
            t = np.arange(steps)*dt
            fi, fo, fo2, fo3, lpu = simulate(G, t, fi)
            
import time
import gc
names = ['DM4DL5_3_ln1', 'DM4DL5_3_ln2', 'DM4DL5_3_ln12']
simulation_traces = {}
for name in names:
    for DM4_b_coeff in [0.0,0.2,0.4,0.6,0.8,1.0]:
        for DL5_b_coeff in [0.0,0.2,0.4,0.6,0.8,1.0]:
            time.sleep(1)
            conc = 10.
            G_path = '/mnt/server-home/mehmet/alclustering/{}.gexf'.format(name)
            visual_components = np.load('/mnt/server-home/mehmet/alclustering/visual_components_{}.npy'.format(name), allow_pickle=True)
            visual_neurons = np.load('/mnt/server-home/mehmet/alclustering/visual_neurons_{}.npy'.format(name), allow_pickle=True).item()
            simulate, G, fi = large_scale_sim(name, G_path, conc=conc, DM4_b = DM4_b_coeff * 0.3, DM4_d = 2.94, DL5_b = DL5_b_coeff * 0.3, exp_name = '_my_experiment_' + str(DM4_b_coeff) + '_' + str(DL5_b_coeff))
            dt = 1e-5
            dur = 4.
            steps = int((dur+dt/2)//dt)
            t = np.arange(steps)*dt
            fi, fo, fo2, fo3, lpu = simulate(G, t, fi)